In [63]:
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Polygon
import ast
import altair as alt


df = pd.read_csv("data/processed/processed_municip_kWh_with_polygons.csv", encoding="utf-8", index_col=0)
df = df.reset_index()		
df

,Province,Municipality,Month,South-facing with vertical (90 degrees) tilt,South-facing with latitude tilt,South-facing with tilt=latitude+15 degrees,South-facing with tilt=latitude-15 degrees,2-axis tracking,Horizontal (0 degree),address,latitude,longitude,polygons
0,Alberta,Acadia Valley,Annual,3.85,4.95,4.71,4.94,7.09,3.73,"Acadia Valley, Alberta",51.158676,-110.210332,"POLYGON ((-110.2141432 51.1587175, -110.211333..."
1,Alberta,Acme,Annual,3.72,4.76,4.53,4.76,6.77,3.62,"Acme, Alberta",51.497782,-113.513721,"POLYGON ((-113.5143844 51.4985257, -113.514325..."
2,Alberta,Airdrie,Annual,3.66,4.70,4.47,4.70,6.67,3.61,"Airdrie, Alberta",51.285970,-114.010620,"POLYGON ((-114.0107674 51.2860953, -114.010766..."
3,Alberta,Alberta Beach,Annual,3.52,4.44,4.21,4.46,6.35,3.42,"Alberta Beach, Alberta",53.674589,-114.357227,"POLYGON ((-114.3570561 53.6745442, -114.356754..."
4,Alberta,Alder Flats,Annual,3.47,4.41,4.18,4.42,6.28,3.44,"Alder Flats, Alberta",52.931756,-114.959071,"POLYGON ((-114.9682284 52.9313586, -114.967447..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3501,Yukon Territory,Swift River,Annual,2.67,3.34,3.07,3.43,4.79,2.76,"Swift River, Yukon Territory",60.001936,-131.189337,"POLYGON ((-131.1884864 60.0048171, -131.189732..."
3502,Yukon Territory,Tagish,Annual,2.79,3.49,3.22,3.58,4.99,2.80,"Tagish, Yukon Territory",60.314831,-134.271491,NaN
3503,Yukon Territory,Teslin,Annual,2.74,3.43,3.16,3.52,4.90,2.80,"Teslin, Yukon Territory",60.165951,-132.724877,"POLYGON ((-132.7251818 60.1660389, -132.725022..."
3504,Yukon Territory,Watson Lake,Annual,2.80,3.46,3.20,3.54,5.01,2.82,"Watson Lake, Yukon Territory",60.062431,-128.706804,"POLYGON ((-128.7031599 60.0618209, -128.705457..."


In [64]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon

# Assuming df is your DataFrame from the CSV file
# Adjust the convert_polygons function to skip non-string polygon data
def convert_polygons(polygon_str):
    if pd.isna(polygon_str) or not isinstance(polygon_str, str):
        return None  # Skip this row if the value is NaN or not a string
    try:
        # Convert the string to a shapely Polygon object using WKT
        return wkt.loads(polygon_str)
    except Exception as e:
        # Print the error and return None for this row
        print(f"Error converting polygon: {e}")
        return None

# Apply the conversion function to the 'polygons' column
df['geometry'] = df['polygons'].apply(convert_polygons)

# Drop rows where polygons conversion failed (if any)
df = df.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Save to GeoJSON
json_file_path = 'data/processed/kWh_poly.json'  # Update with your correct path
gdf.to_file(json_file_path, driver='GeoJSON')